In [1]:
from datetime import datetime, timedelta
import pandas as pd

from feast import FeatureStore

In [2]:
!feast

Usage: feast [OPTIONS] COMMAND [ARGS]...

  Feast CLI

  For more information, see our public docs at https://docs.feast.dev/

  For any questions, you can reach us at https://slack.feast.dev/

Options:
  -c, --chdir TEXT  Switch to a different feature repository directory before
                    executing the given subcommand.
  --log-level TEXT  The logging level. One of DEBUG, INFO, WARNING, ERROR, and
                    CRITICAL (case-insensitive).
  --help            Show this message and exit.

Commands:
  alpha                    Access alpha features
  apply                    Create or update a feature store deployment
  endpoint                 Display feature server endpoints.
  entities                 Access entities
  feature-services         Access feature services
  feature-views            Access feature views
  init                     Create a new Feast repository
  materialize              Run a (non-incremental) materialization job to...
  materialize-increment

In [3]:
!feast endpoint

Can't find feature_store.yaml at /Users/avkash/work/prodramp/publiccode/feature-store/feast-starter. Make sure you're running feast from an initialized feast repository. 


In [4]:
!pwd

/Users/avkash/work/prodramp/publiccode/feature-store/feast-starter


In [5]:
!ls

1-Install-init.ipynb                 5-Prediction-stored-features.ipynb
2-Using Existing Feature Store.ipynb Access-sqlite-db.ipynb
3-Understand Data.ipynb              driver_orders.csv
4-Working-with-features.ipynb        feature_repo


In [6]:
%cd feature_repo

/Users/avkash/work/prodramp/publiccode/feature-store/feast-starter/feature_repo


In [7]:
!pwd

/Users/avkash/work/prodramp/publiccode/feature-store/feast-starter/feature_repo


In [8]:
!feast endpoint

/Users/avkash/anaconda3/lib/python3.7/site-packages/jinja2/utils.py:485: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping
/Users/avkash/anaconda3/lib/python3.7/site-packages/jinja2/runtime.py:318: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
02/23/2022 02:34:57 PM INFO:There is no active feature server.


## Step 3: Register feature definitions and deploy your feature store
- The apply command scans python files in the current directory for feature view/entity definitions, registers the objects, and deploys infrastructure. 
- In this example, it reads example.py (shown again below for convenience) and sets up SQLite online store tables. 

Note that we had specified SQLite as the default online store by using the local provider in feature_store.yaml.
feast apply


In [ ]:
!ls

In [9]:
!cat example.py

# This is an example feature definition file

from google.protobuf.duration_pb2 import Duration

from feast import Entity, Feature, FeatureView, FileSource, ValueType

# Read data from parquet files. Parquet is convenient for local development mode. For
# production, you can use your favorite DWH, such as BigQuery. See Feast documentation
# for more info.
driver_hourly_stats = FileSource(
    path="/Users/avkash/work/prodramp/publiccode/feature-store/feast-starter/feature_repo/data/driver_stats.parquet",
    event_timestamp_column="event_timestamp",
    created_timestamp_column="created",
)

# Define an entity for the driver. You can think of entity as a primary key used to
# fetch features.
driver = Entity(name="driver_id", value_type=ValueType.INT64, description="driver id",)

# Our parquet files contain sample data that includes a driver_id column, timestamps and
# three feature column. Here we define a Feature View that will allow us to serve this
# data to our model online.
driver

In [10]:
from google.protobuf.duration_pb2 import Duration
from feast import Entity, Feature, FeatureView, FileSource, ValueType

In [11]:
# Read data from parquet files. Parquet is convenient for local development mode. For
# production, you can use your favorite DWH, such as BigQuery. See Feast documentation
# for more info.
driver_hourly_stats = FileSource(
    path="/Users/avkash/work/prodramp/publiccode/feature-store/feast-starter/feature_repo/data/driver_stats.parquet",
    event_timestamp_column="event_timestamp",
    created_timestamp_column="created",
)


In [12]:
driver_hourly_stats

In [13]:
# Define an entity for the driver. You can think of entity as a primary key used to
# fetch features.
driver = Entity(name="driver_id", value_type=ValueType.INT64, description="driver id",)

In [14]:
driver

In [15]:
!feast entities list

/Users/avkash/anaconda3/lib/python3.7/site-packages/jinja2/utils.py:485: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping
/Users/avkash/anaconda3/lib/python3.7/site-packages/jinja2/runtime.py:318: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
NAME    DESCRIPTION    TYPE


In [16]:
# Our parquet files contain sample data that includes a driver_id column, timestamps and
# three feature column. Here we define a Feature View that will allow us to serve this
# data to our model online.
driver_hourly_stats_view = FeatureView(
    name="driver_hourly_stats",
    entities=["driver_id"],
    ttl=Duration(seconds=86400 * 1),
    features=[
        Feature(name="conv_rate", dtype=ValueType.FLOAT),
        Feature(name="acc_rate", dtype=ValueType.FLOAT),
        Feature(name="avg_daily_trips", dtype=ValueType.INT64),
    ],
    online=True,
    batch_source=driver_hourly_stats,
    tags={},
)

In [17]:
driver_hourly_stats_view

<FeatureView(_name = driver_hourly_stats, _features = [conv_rate-ValueType.FLOAT, acc_rate-ValueType.FLOAT, avg_daily_trips-ValueType.INT64], _projection = FeatureViewProjection(name='driver_hourly_stats', name_alias=None, features=[conv_rate-ValueType.FLOAT, acc_rate-ValueType.FLOAT, avg_daily_trips-ValueType.INT64], join_key_map={}), created_timestamp = None, last_updated_timestamp = None, entities = ['driver_id'], tags = {}, ttl = 1 day, 0:00:00, online = True, input = <feast.infra.offline_stores.file_source.FileSource object at 0x7fb1f8a873d0>, batch_source = <feast.infra.offline_stores.file_source.FileSource object at 0x7fb1f8a873d0>, stream_source = None, materialization_intervals = [])>

In [18]:
driver_hourly_stats_view.entities

['driver_id']

In [19]:
driver_hourly_stats_view.input

In [20]:
driver_hourly_stats_view.features

[conv_rate-ValueType.FLOAT,
 acc_rate-ValueType.FLOAT,
 avg_daily_trips-ValueType.INT64]

In [21]:
!ls

__init__.py        data               example.py         feature_store.yaml


In [22]:
!cat feature_store.yaml

project: feature_repo
registry: data/registry.db
provider: local
online_store:
    path: data/online_store.db

In [23]:
!python example.py

In [24]:
!feast endpoint

/Users/avkash/anaconda3/lib/python3.7/site-packages/jinja2/utils.py:485: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping
/Users/avkash/anaconda3/lib/python3.7/site-packages/jinja2/runtime.py:318: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
02/23/2022 02:37:32 PM INFO:There is no active feature server.


In [25]:
!feast apply

/Users/avkash/anaconda3/lib/python3.7/site-packages/jinja2/utils.py:485: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping
/Users/avkash/anaconda3/lib/python3.7/site-packages/jinja2/runtime.py:318: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
Created entity driver_id
Created feature view driver_hourly_stats

Created sqlite table feature_repo_driver_hourly_stats



In [26]:
!ls -la

total 16
drwxr-xr-x   6 avkash  staff   192 Feb 23 14:19 .
drwxr-xr-x  11 avkash  staff   352 Feb 23 14:37 ..
-rw-r--r--   1 avkash  staff     0 Feb 20 09:04 __init__.py
drwxr-xr-x   5 avkash  staff   160 Feb 23 14:37 data
-rw-r--r--   1 avkash  staff  1402 Feb 23 14:19 example.py
-rw-r--r--   1 avkash  staff   109 Feb 23 14:19 feature_store.yaml


In [27]:
!ls -la data

total 112
drwxr-xr-x  5 avkash  staff    160 Feb 23 14:37 .
drwxr-xr-x  6 avkash  staff    192 Feb 23 14:19 ..
-rw-r--r--  1 avkash  staff  34468 Feb 23 14:19 driver_stats.parquet
-rw-r--r--  1 avkash  staff  16384 Feb 23 14:37 online_store.db
-rw-r--r--  1 avkash  staff    732 Feb 23 14:37 registry.db
